# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df_to_load = "../WeatherPy/weather_df.csv"
weather_data = pd.read_csv(weather_df_to_load)
dropna_data = weather_data.dropna()
del dropna_data["Unnamed: 0"]
dropna_data.head(10)


,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,Port Elizabeth,ZA,1601529996,-33.92,25.57,75,87,59.00,11.41
1,Bethel,US,1601530283,41.37,-73.41,1,63,60.80,9.17
2,Bluff,NZ,1601530284,-46.60,168.33,84,56,60.01,4.00
3,Saldanha,ZA,1601530286,-33.01,17.94,7,71,51.80,14.99
4,Pokhara,NP,1601530287,28.23,83.98,3,62,82.00,5.79
5,Richards Bay,ZA,1601530225,-28.78,32.04,0,67,73.22,10.45
6,Mataura,NZ,1601530291,-46.19,168.86,85,56,60.01,4.00
7,Stade,DE,1601530292,53.60,9.48,19,93,53.01,3.36
8,Punta Arenas,CL,1601530294,-53.15,-70.92,20,86,28.40,12.75
9,Rikitea,PF,1601530296,-23.12,-134.97,73,81,72.95,25.48


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

loc = dropna_data[["Latitude", "Longitude"]]
hum = dropna_data["Humidity (%)"].astype(float)


In [ ]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Set cities to fit weather conditions
vacation_zero_cloud = dropna_data[dropna_data["Cloudiness (%)"] == 0]
vacation_above_70_degrees = vacation_zero_cloud[vacation_zero_cloud["Max Temp (F)"] > 70]
vacation_below_80_degrees = vacation_above_70_degrees[vacation_above_70_degrees["Max Temp (F)"] < 80]
vacation_slow_wind = vacation_below_80_degrees[vacation_below_80_degrees["Wind Speed (mph)"] < 10]
perfect_vacation = vacation_slow_wind[vacation_slow_wind["Humidity (%)"] < 60]

#Set index
hotel_df = perfect_vacation.reset_index()
del hotel_df["index"]
hotel_df

vacation_locations = hotel_df[["Latitude", "Longitude"]]
vacation_humidity = hotel_df["Humidity (%)"].astype(float)

# Plot Heatmap
vacation_fig = gmaps.figure()

# Create heat layer
vacation_heat_layer = gmaps.heatmap_layer(vacation_locations, weights=vacation_humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2.5)

# Add layer
vacation_fig.add_layer(vacation_heat_layer)

# Display figure
vacation_fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel = []
x = 0
#loop
for cities in hotel_df['City']:
    target_coordinates = str(hotel_df["Latitude"][x])+' , '+str(hotel_df["Longitude"][x])
    x+=1
    target_radius = 5000
    target_type = 'lodging'
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
   
    try:
        response = requests.get(url, params=params)
        hotel_results = response.json()
        hotel.append(hotel_results['results'][0]['name'])
        
    except IndexError:
        hotel.append(np.nan)

#append dataframe with hotel names
hotel_df['Hotel Name'] = hotel
hotel_df

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Hotel Name
0,San Quintín,MX,1601529753,30.48,-115.95,0,39,79.54,5.86,NaN
1,Āsmār,AF,1601530590,35.03,71.36,0,25,77.76,2.75,NaN
2,Chaman,PK,1601530740,30.92,66.45,0,17,70.70,5.41,NaN
3,Rusape,ZW,1601530771,-18.53,32.13,0,41,72.99,7.40,NaN
4,Guerrero Negro,MX,1601530797,27.98,-114.06,0,50,78.71,6.51,NaN
5,Sīrjān,IR,1601531084,29.45,55.68,0,21,74.01,8.32,NaN


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotels in hotel])
vacation_fig.add_layer(markers)

# Display figure
vacation_fig

Figure(layout=FigureLayout(height='420px'))